<a href="https://colab.research.google.com/github/androbaza/my-kaggle-submissions/blob/master/WiFi%20Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import random
from random import sample
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import gc
import torch.nn as nn
from torch.utils.data import Dataset
import torch
import matplotlib.pyplot as plt
from scipy.ndimage.filters import uniform_filter1d
from scipy.interpolate import interp1d
import numpy as np 
import pandas as pd  
import torch.nn as nn
import os
import time
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from collections import defaultdict

EPOCH = 50
BATCH_SIZE = 256
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MEMORY_LENGTH = 5 # max sequence length
WIFI_NUM = 50
FOLDS = 10
inpath = '/content/drive/MyDrive/wifi-loc/'
# metapath = inpath + 'metadata'
trainpath = inpath + 'train'
testpath = inpath + 'test'

def set_seed(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True

set_seed()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def mean_error_dist(x_pred, y_pred, z_pred, x_true, y_true, z_true):
    dist = 0
    for index in range(len(x_pred)):
        dist = dist + np.linalg.norm(np.array((x_pred[index], y_pred[index], z_pred[index])) - np.array((x_true[index], y_true[index], z_true[index])))
    return dist / len(x_true)

# LightBGM 2.97

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os
import json
import glob
import cv2
import numpy as np
import lightgbm as lgb
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.multioutput import RegressorChain

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil

Mounted at /content/drive


In [ ]:
train_dir = "/content/drive/MyDrive/wifi-loc/train/*.csv"
test_dir = "/content/drive/MyDrive/wifi-loc/test/*.csv"

# Paths for train & test files
train_paths = sorted(glob.glob(train_dir))
test_paths = sorted(glob.glob(test_dir))

print("Len Train Files: {}".format(len(train_paths)), "\n" + "Len Test Files: {}".format(len(test_paths)))

Len Train Files: 80 
Len Test Files: 11


In [ ]:
train_df = pd.concat((pd.read_csv(f) for f in train_paths))
test_df = pd.concat((pd.read_csv(f) for f in test_paths))

test_df.drop(test_df.columns[-12:-6], axis=1, inplace=True)
train_df.drop(train_df.columns[-12:-6], axis=1, inplace=True)

In [ ]:
def train_lgbm1():

    # --- Read in data ---
    # train_df = train_df.sample(frac=1, random_state=42)
    
    # train_df = pd.concat((pd.read_csv(f) for f in train_paths))
    # test_df = pd.concat((pd.read_csv(f) for f in test_paths))

    # Drop unneeded columns
    # test_df.drop(test_df.columns[-12:-6], axis=1, inplace=True)
    # train_df.drop(train_df.columns[-12:-6], axis=1, inplace=True)
    # test_df = test_df.iloc[:, :-3]

    # --- Data Validation ---
    # Train features + targets
    X = train_df.iloc[:, :-3]
    y = train_df.iloc[:, -3:]

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    train_data_x = lgb.Dataset(X_train, label=y_train.iloc[:, -3])
    valid_data_x = lgb.Dataset(X_valid, label=y_valid.iloc[:, -3], reference=train_data_x)

    train_data_y = lgb.Dataset(X_train, label=y_train.iloc[:, -2])
    valid_data_y = lgb.Dataset(X_valid, label=y_valid.iloc[:, -2], reference=train_data_y)

    train_data_z = lgb.Dataset(X_train, label=y_train.iloc[:, -1])
    valid_data_z = lgb.Dataset(X_valid, label=y_valid.iloc[:, -1], reference=train_data_z)

    SEARCH_PARAMS = {'learning_rate': 0.5,
                    #  'min_data_in_leaf': 500,
                     'max_bin ': 2000,
                     'num_iterations': 30,
                     'num_leaves': 2560,
                     'max_depth': 35,
                     # 'subsample': 0.2,
                     # 'feature_fraction': 0.8, 
                    }

    FIXED_PARAMS={'objective': 'regression_l1',
                'boosting':'dart',
                 }

    params = {**FIXED_PARAMS,**SEARCH_PARAMS}

    # --- Model Training ---
    model_x = lgb.train(params, train_data_x, valid_sets=valid_data_x)

    model_y = lgb.train(params, train_data_y, valid_sets=valid_data_y)

    model_z = lgb.train(params, train_data_z, valid_sets=valid_data_z)
    
    lgbm_predictions = pd.DataFrame(columns=['x', 'y', 'z'])
    lgbm_predictions.iloc[:, 0] = model_x.predict(test_df.iloc[:, :-3])
    lgbm_predictions.iloc[:, 1] = model_y.predict(test_df.iloc[:, :-3])
    lgbm_predictions.iloc[:, 2] = model_z.predict(test_df.iloc[:, :-3])

    return model_x, model_y, model_z, lgbm_predictions

In [ ]:
model_x, model_y, model_z, lgbm_predictions = train_lgbm1()

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `num_iterations` in params. Will use it instead of argument



[1]	valid_0's l1: 9.59254
[2]	valid_0's l1: 5.64686
[3]	valid_0's l1: 3.55557
[4]	valid_0's l1: 2.48743
[5]	valid_0's l1: 1.75215
[6]	valid_0's l1: 1.22484
[7]	valid_0's l1: 0.905466
[8]	valid_0's l1: 1.46438
[9]	valid_0's l1: 1.06678
[10]	valid_0's l1: 0.834572
[11]	valid_0's l1: 0.654417
[12]	valid_0's l1: 0.762583
[13]	valid_0's l1: 0.593082
[14]	valid_0's l1: 0.469746
[15]	valid_0's l1: 0.386801
[16]	valid_0's l1: 0.33198
[17]	valid_0's l1: 0.298123
[18]	valid_0's l1: 0.272692
[19]	valid_0's l1: 0.255573
[20]	valid_0's l1: 0.242294
[21]	valid_0's l1: 0.294778
[22]	valid_0's l1: 0.255084
[23]	valid_0's l1: 0.235591
[24]	valid_0's l1: 0.223605
[25]	valid_0's l1: 0.214546
[26]	valid_0's l1: 0.208603
[27]	valid_0's l1: 0.203736
[28]	valid_0's l1: 0.216976
[29]	valid_0's l1: 0.20669
[30]	valid_0's l1: 0.200747
[1]	valid_0's l1: 4.28636
[2]	valid_0's l1: 2.78199
[3]	valid_0's l1: 1.75525
[4]	valid_0's l1: 1.17884
[5]	valid_0's l1: 0.817611
[6]	valid_0's l1: 0.54292
[7]	valid_0's l1: 0.40

In [ ]:
MED = mean_error_dist(lgbm_predictions.iloc[:, -3].to_numpy(), lgbm_predictions.iloc[:, -2].to_numpy(), lgbm_predictions.iloc[:, -1].to_numpy(), test_df.iloc[:, -3].to_numpy(), test_df.iloc[:, -2].to_numpy(), test_df.iloc[:, -1].to_numpy())

In [ ]:
MED

2.9721766747370624

# LBGM#2 3.44

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os
import json
import glob
import cv2
import numpy as np
import lightgbm as lgb
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.multioutput import RegressorChain

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil

Mounted at /content/drive


In [ ]:
train_dir = "/content/drive/MyDrive/wifi-loc/train/*.csv"
test_dir = "/content/drive/MyDrive/wifi-loc/test/*.csv"

# Paths for train & test files
train_paths = sorted(glob.glob(train_dir))
test_paths = sorted(glob.glob(test_dir))

print("Len Train Files: {}".format(len(train_paths)), "\n" + "Len Test Files: {}".format(len(test_paths)))

Len Train Files: 82 
Len Test Files: 9


In [ ]:
train_df = pd.concat((pd.read_csv(f) for f in train_paths))
test_df = pd.concat((pd.read_csv(f) for f in test_paths))

test_df.drop(test_df.columns[-12:-6], axis=1, inplace=True)
train_df.drop(train_df.columns[-12:-6], axis=1, inplace=True)

In [ ]:
def train_lgbm2():

    # --- Read in data ---
    # train_df = train_df.sample(frac=1, random_state=42)
    
    # train_df = pd.concat((pd.read_csv(f) for f in train_paths))
    # test_df = pd.concat((pd.read_csv(f) for f in test_paths))

    # Drop unneeded columns
    # test_df.drop(test_df.columns[-12:-6], axis=1, inplace=True)
    # train_df.drop(train_df.columns[-12:-6], axis=1, inplace=True)
    # test_df = test_df.iloc[:, :-3]

    # --- Data Validation ---
    # Train features + targets
    X = train_df.iloc[:, :-3]
    y = train_df.iloc[:, -3:]

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

    train_data_x = lgb.Dataset(X_train, label=y_train.iloc[:, -3])
    valid_data_x = lgb.Dataset(X_valid, label=y_valid.iloc[:, -3], reference=train_data_x)

    train_data_y = lgb.Dataset(X_train, label=y_train.iloc[:, -2])
    valid_data_y = lgb.Dataset(X_valid, label=y_valid.iloc[:, -2], reference=train_data_y)

    train_data_z = lgb.Dataset(X_train, label=y_train.iloc[:, -1])
    valid_data_z = lgb.Dataset(X_valid, label=y_valid.iloc[:, -1], reference=train_data_z)

    SEARCH_PARAMS = {'learning_rate': 0.1,
                    #  'min_data_in_leaf': 500,
                     'max_bin ': 2000,
                     'num_iterations': 35,
                     'num_leaves': 2560,
                     'max_depth': 35,
                     # 'subsample': 0.2,
                     # 'feature_fraction': 0.8, 
                    }

    FIXED_PARAMS={'objective': 'regression_l1',
                'boosting':'dart',
                 }

    params = {**FIXED_PARAMS,**SEARCH_PARAMS}

    # --- Model Training ---
    model_x = lgb.train(params, train_data_x, valid_sets=valid_data_x)

    model_y = lgb.train(params, train_data_y, valid_sets=valid_data_y)

    model_z = lgb.train(params, train_data_z, valid_sets=valid_data_z)
    
    lgbm_predictions = pd.DataFrame(columns=['x', 'y', 'z'])
    lgbm_predictions.iloc[:, 0] = model_x.predict(test_df.iloc[:, :-3])
    lgbm_predictions.iloc[:, 1] = model_y.predict(test_df.iloc[:, :-3])
    lgbm_predictions.iloc[:, 2] = model_z.predict(test_df.iloc[:, :-3])

    return model_x, model_y, model_z, lgbm_predictions

In [ ]:
model_x, model_y, model_z, lgbm_predictions = train_lgbm2()

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `num_iterations` in params. Will use it instead of argument



[1]	valid_0's l1: 15.8115
[2]	valid_0's l1: 14.3171
[3]	valid_0's l1: 12.9898
[4]	valid_0's l1: 11.777
[5]	valid_0's l1: 10.638
[6]	valid_0's l1: 9.61581
[7]	valid_0's l1: 8.75802
[8]	valid_0's l1: 9.23351
[9]	valid_0's l1: 8.43015
[10]	valid_0's l1: 7.69122
[11]	valid_0's l1: 7.03236
[12]	valid_0's l1: 7.3455
[13]	valid_0's l1: 6.68427
[14]	valid_0's l1: 6.11603
[15]	valid_0's l1: 5.60915
[16]	valid_0's l1: 5.14771
[17]	valid_0's l1: 4.73165
[18]	valid_0's l1: 4.3489
[19]	valid_0's l1: 4.00488
[20]	valid_0's l1: 3.69295
[21]	valid_0's l1: 3.89187
[22]	valid_0's l1: 3.59247
[23]	valid_0's l1: 3.31769
[24]	valid_0's l1: 3.06497
[25]	valid_0's l1: 2.82004
[26]	valid_0's l1: 2.59674
[27]	valid_0's l1: 2.39825
[28]	valid_0's l1: 2.53745
[29]	valid_0's l1: 2.35736
[30]	valid_0's l1: 2.18614
[31]	valid_0's l1: 2.3051
[32]	valid_0's l1: 2.14869
[33]	valid_0's l1: 1.99537
[34]	valid_0's l1: 1.85183
[35]	valid_0's l1: 1.9939
[1]	valid_0's l1: 6.41163
[2]	valid_0's l1: 5.92752
[3]	valid_0's l1: 

In [ ]:
test_df = pd.concat((pd.read_csv(f) for f in test_paths))
MED = mean_error_dist(lgbm_predictions.iloc[:, -3].to_numpy(), lgbm_predictions.iloc[:, -2].to_numpy(), lgbm_predictions.iloc[:, -1].to_numpy(), test_df.iloc[:, -3].to_numpy(), test_df.iloc[:, -2].to_numpy(), test_df.iloc[:, -1].to_numpy())

In [ ]:
MED

3.4497551153870165

# LGBM#3

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

import os
import json
import glob
import cv2
import numpy as np
import lightgbm as lgb
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.multioutput import RegressorChain

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil

Mounted at /content/drive


In [ ]:
train_dir = "/content/drive/MyDrive/wifi-loc/train/*.csv"
test_dir = "/content/drive/MyDrive/wifi-loc/test/*.csv"

# Paths for train & test files
train_paths = sorted(glob.glob(train_dir))
test_paths = sorted(glob.glob(test_dir))

print("Len Train Files: {}".format(len(train_paths)), "\n" + "Len Test Files: {}".format(len(test_paths)))

Len Train Files: 83 
Len Test Files: 8


In [ ]:
train_df = pd.concat((pd.read_csv(f) for f in train_paths))
test_df = pd.concat((pd.read_csv(f) for f in test_paths))

In [ ]:
def train_lgbm3():

    # --- Read in data ---
    # train_df = train_df.sample(frac=1, random_state=42)
    
    # train_df = pd.concat((pd.read_csv(f) for f in train_paths))
    # test_df = pd.concat((pd.read_csv(f) for f in test_paths))

    # Drop unneeded columns
    test_df.drop(test_df.columns[-12:-6], axis=1, inplace=True)
    train_df.drop(train_df.columns[-12:-6], axis=1, inplace=True)
    # test_df = test_df.iloc[:, :-3]

    # --- Data Validation ---
    # Train features + targets
    X = train_df.iloc[:, :-3]
    y = train_df.iloc[:, -3:]

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    train_data_x = lgb.Dataset(X_train, label=y_train.iloc[:, -3])
    valid_data_x = lgb.Dataset(X_valid, label=y_valid.iloc[:, -3], reference=train_data_x)

    train_data_y = lgb.Dataset(X_train, label=y_train.iloc[:, -2])
    valid_data_y = lgb.Dataset(X_valid, label=y_valid.iloc[:, -2], reference=train_data_y)

    train_data_z = lgb.Dataset(X_train, label=y_train.iloc[:, -1])
    valid_data_z = lgb.Dataset(X_valid, label=y_valid.iloc[:, -1], reference=train_data_z)

    SEARCH_PARAMS = {'learning_rate': 0.4,
                    #  'min_data_in_leaf': 500,
                     'max_bin ': 1000,
                     'num_iterations': 40,
                     'num_leaves': 1000,
                     'max_depth': 25,
                     # 'subsample': 0.2,
                     # 'feature_fraction': 0.8, 
                    }

    FIXED_PARAMS={'objective': 'regression_l1',
                'boosting':'dart',
                 }

    params = {**FIXED_PARAMS,**SEARCH_PARAMS}

    # --- Model Training ---
    model_x = lgb.train(params, train_data_x, valid_sets=valid_data_x)

    model_y = lgb.train(params, train_data_y, valid_sets=valid_data_y)

    model_z = lgb.train(params, train_data_z, valid_sets=valid_data_z)
    
    lgbm_predictions = pd.DataFrame(columns=['x', 'y', 'z'])
    lgbm_predictions.iloc[:, 0] = model_x.predict(test_df.iloc[:, :-3])
    lgbm_predictions.iloc[:, 1] = model_y.predict(test_df.iloc[:, :-3])
    lgbm_predictions.iloc[:, 2] = model_z.predict(test_df.iloc[:, :-3])

    return model_x, model_y, model_z, lgbm_predictions

In [ ]:
model_x, model_y, model_z, lgbm_predictions = train_lgbm3()

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `num_iterations` in params. Will use it instead of argument



[1]	valid_0's l1: 10.8929
[2]	valid_0's l1: 7.15075
[3]	valid_0's l1: 4.89042
[4]	valid_0's l1: 3.47761
[5]	valid_0's l1: 2.60381
[6]	valid_0's l1: 1.96262
[7]	valid_0's l1: 1.54417
[8]	valid_0's l1: 2.14322
[9]	valid_0's l1: 1.68768
[10]	valid_0's l1: 1.33635
[11]	valid_0's l1: 1.0495
[12]	valid_0's l1: 1.20263
[13]	valid_0's l1: 0.986054
[14]	valid_0's l1: 0.809911
[15]	valid_0's l1: 0.67749
[16]	valid_0's l1: 0.59151
[17]	valid_0's l1: 0.525025
[18]	valid_0's l1: 0.473176
[19]	valid_0's l1: 0.434865
[20]	valid_0's l1: 0.406194
[21]	valid_0's l1: 0.46143
[22]	valid_0's l1: 0.414101
[23]	valid_0's l1: 0.380254
[24]	valid_0's l1: 0.353183
[25]	valid_0's l1: 0.329543
[26]	valid_0's l1: 0.312951
[27]	valid_0's l1: 0.298067
[28]	valid_0's l1: 0.315133
[29]	valid_0's l1: 0.29868
[30]	valid_0's l1: 0.285224
[31]	valid_0's l1: 0.299968
[32]	valid_0's l1: 0.284105
[33]	valid_0's l1: 0.272331
[34]	valid_0's l1: 0.264221
[35]	valid_0's l1: 0.274814
[36]	valid_0's l1: 0.386477
[37]	valid_0's l1:

In [ ]:
# test_df = pd.concat((pd.read_csv(f) for f in test_paths))
MED = mean_error_dist(lgbm_predictions.iloc[:, -3].to_numpy(), lgbm_predictions.iloc[:, -2].to_numpy(), lgbm_predictions.iloc[:, -1].to_numpy(), test_df.iloc[:, -3].to_numpy(), test_df.iloc[:, -2].to_numpy(), test_df.iloc[:, -1].to_numpy())

In [ ]:
MED

6.430963510986318

# LBGM floor classification

# LightBGM 2.97

In [1]:
from google.colab import drive
drive.mount("/content/drive")

import os
import json
import glob
import cv2
import numpy as np
import lightgbm as lgb
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.multioutput import RegressorChain

from PIL import Image, ImageOps
from skimage import io
from skimage.color import rgba2rgb, rgb2xyz
from tqdm import tqdm
from dataclasses import dataclass
from math import floor, ceil

Mounted at /content/drive


In [2]:
train_dir = "/content/drive/MyDrive/wifi-loc/train/*.csv"
test_dir = "/content/drive/MyDrive/wifi-loc/test/*.csv"

# Paths for train & test files
train_paths = sorted(glob.glob(train_dir))
test_paths = sorted(glob.glob(test_dir))

print("Len Train Files: {}".format(len(train_paths)), "\n" + "Len Test Files: {}".format(len(test_paths)))

Len Train Files: 82 
Len Test Files: 9


In [3]:
train_df = pd.concat((pd.read_csv(f) for f in train_paths))
test_df = pd.concat((pd.read_csv(f) for f in test_paths))

test_df.drop(test_df.columns[-12:-6], axis=1, inplace=True)
train_df.drop(train_df.columns[-12:-6], axis=1, inplace=True)

In [ ]:
def train_lgbm1():

    # --- Read in data ---
    # train_df = train_df.sample(frac=1, random_state=42)
    
    # train_df = pd.concat((pd.read_csv(f) for f in train_paths))
    # test_df = pd.concat((pd.read_csv(f) for f in test_paths))

    # Drop unneeded columns
    # test_df.drop(test_df.columns[-12:-6], axis=1, inplace=True)
    # train_df.drop(train_df.columns[-12:-6], axis=1, inplace=True)
    # test_df = test_df.iloc[:, :-3]

    # --- Data Validation ---
    # Train features + targets
    X = train_df.iloc[:, :-3]
    y = train_df.iloc[:, -3:]

    # First train for floors and predict floors. add that data for testing for x and y

    

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    train_data_x = lgb.Dataset(X_train, label=y_train.iloc[:, -3])
    valid_data_x = lgb.Dataset(X_valid, label=y_valid.iloc[:, -3], reference=train_data_x)

    train_data_y = lgb.Dataset(X_train, label=y_train.iloc[:, -2])
    valid_data_y = lgb.Dataset(X_valid, label=y_valid.iloc[:, -2], reference=train_data_y)

    train_data_z = lgb.Dataset(X_train, label=y_train.iloc[:, -1])
    valid_data_z = lgb.Dataset(X_valid, label=y_valid.iloc[:, -1], reference=train_data_z)

    SEARCH_PARAMS = {'learning_rate': 0.5,
                    #  'min_data_in_leaf': 500,
                     'max_bin ': 2000,
                     'num_iterations': 30,
                     'num_leaves': 2560,
                     'max_depth': 35,
                     # 'subsample': 0.2,
                     # 'feature_fraction': 0.8, 
                    }

    FIXED_PARAMS={'objective': 'regression_l1',
                'boosting':'dart',
                 }

    params = {**FIXED_PARAMS,**SEARCH_PARAMS}

    # --- Model Training ---
    model_x = lgb.train(params, train_data_x, valid_sets=valid_data_x)

    model_y = lgb.train(params, train_data_y, valid_sets=valid_data_y)

    model_z = lgb.train(params, train_data_z, valid_sets=valid_data_z)
    
    lgbm_predictions = pd.DataFrame(columns=['x', 'y', 'z'])
    lgbm_predictions.iloc[:, 0] = model_x.predict(test_df.iloc[:, :-3])
    lgbm_predictions.iloc[:, 1] = model_y.predict(test_df.iloc[:, :-3])
    lgbm_predictions.iloc[:, 2] = model_z.predict(test_df.iloc[:, :-3])

    return model_x, model_y, model_z, lgbm_predictions

In [ ]:
model_x, model_y, model_z, lgbm_predictions = train_lgbm1()

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning:

Found `num_iterations` in params. Will use it instead of argument



[1]	valid_0's l1: 9.59254
[2]	valid_0's l1: 5.64686
[3]	valid_0's l1: 3.55557
[4]	valid_0's l1: 2.48743
[5]	valid_0's l1: 1.75215
[6]	valid_0's l1: 1.22484
[7]	valid_0's l1: 0.905466
[8]	valid_0's l1: 1.46438
[9]	valid_0's l1: 1.06678
[10]	valid_0's l1: 0.834572
[11]	valid_0's l1: 0.654417
[12]	valid_0's l1: 0.762583
[13]	valid_0's l1: 0.593082
[14]	valid_0's l1: 0.469746
[15]	valid_0's l1: 0.386801
[16]	valid_0's l1: 0.33198
[17]	valid_0's l1: 0.298123
[18]	valid_0's l1: 0.272692
[19]	valid_0's l1: 0.255573
[20]	valid_0's l1: 0.242294
[21]	valid_0's l1: 0.294778
[22]	valid_0's l1: 0.255084
[23]	valid_0's l1: 0.235591
[24]	valid_0's l1: 0.223605
[25]	valid_0's l1: 0.214546
[26]	valid_0's l1: 0.208603
[27]	valid_0's l1: 0.203736
[28]	valid_0's l1: 0.216976
[29]	valid_0's l1: 0.20669
[30]	valid_0's l1: 0.200747
[1]	valid_0's l1: 4.28636
[2]	valid_0's l1: 2.78199
[3]	valid_0's l1: 1.75525
[4]	valid_0's l1: 1.17884
[5]	valid_0's l1: 0.817611
[6]	valid_0's l1: 0.54292
[7]	valid_0's l1: 0.40

In [ ]:
MED = mean_error_dist(lgbm_predictions.iloc[:, -3].to_numpy(), lgbm_predictions.iloc[:, -2].to_numpy(), lgbm_predictions.iloc[:, -1].to_numpy(), test_df.iloc[:, -3].to_numpy(), test_df.iloc[:, -2].to_numpy(), test_df.iloc[:, -1].to_numpy())

In [ ]:
MED

2.9721766747370624